#### 1. 먼저환율을 예측한다. (예측 정확도에 따라서 점수화 할것.)
#### LSTM, gru, 트렌스포머 등을  사용.


###### 1-1 데이터 전처리

In [5]:
import pandas as pd
import numpy as np
save_path = "C:\\Users\\Happy\\Desktop\\논문용\\krwusdtdata\\전처리된파일\\"

In [6]:
# 인베스트 컴 데이터 전처리 (인베스트에서 csv 파일 다운 후 아예 열지 않을 것을 권장!!)
KRW_USDT = pd.read_csv("USD_KRW 내역.csv") 
KRW_EUR = pd.read_csv("EUR_KRW 내역.csv")
KRW_CNY = pd.read_csv("CNY_KRW 내역.csv")
DXY = pd.read_csv("달러 지수 내역.csv")# 미국 달러 인덱스

KRW_USDT_2 = pd.read_csv("USD_KRW 내역 2.csv") 
KRW_EUR_2 = pd.read_csv("EUR_KRW 내역 2.csv")
KRW_CNY_2 = pd.read_csv("CNY_KRW 내역 2.csv")
DXY_2 = pd.read_csv("달러 지수 내역 2.csv")# 미국 달러 인덱스

KRW_USDT = pd.concat([KRW_USDT,KRW_USDT_2],axis = 0,ignore_index=True)#.reset_index(drop =True,inplace = False)
KRW_EUR = pd.concat([KRW_EUR,KRW_EUR_2],axis = 0,ignore_index=True)#.reset_index(drop =True,inplace = False)
KRW_CNY = pd.concat([KRW_CNY,KRW_CNY_2],axis = 0,ignore_index=True)#.reset_index(drop =True,inplace = False)
DXY = pd.concat([DXY,DXY_2],axis = 0, ignore_index=True)#.reset_index(drop =True,inplace = False)

def Pretreatment_by_invest(df): #인베스트에서 다운한 데이터 전처리
    df = df.rename(columns={'오픈':'Open','고가':'High',"저가":"Low","종가":"Close"})
    df = df[["날짜","Open","High","Low","Close"]]
    # 날짜 daytime형태로 변환
    df["날짜"] = [df["날짜"][i][0:4]+"-"+df["날짜"][i][6:8]+"-"+df["날짜"][i][10:12] for i in range(len(df["날짜"]))]
    df['날짜'] = pd.to_datetime(df['날짜'], format='%Y-%m-%d', errors='raise')
    
    columns = df[["Open","High","Low","Close"]].columns
    try:
        for i in range(len(columns)):
            df[columns[i]] = df[columns[i]].apply(lambda x : float(x.replace(',', ''))) ## @ lambda를 사용하여 한번에 모든 행 적용
    except:
        pass
    
    df = df.sort_values(['날짜'],ascending= True).drop_duplicates('날짜').reset_index(drop = True).rename(columns={'날짜':'time'})
    
    return df


KRW_USDT = Pretreatment_by_invest(KRW_USDT)
KRW_EUR = Pretreatment_by_invest(KRW_EUR)
KRW_CNY = Pretreatment_by_invest(KRW_CNY)
DXY = Pretreatment_by_invest(DXY)

In [7]:
KRW_USDT.to_csv(save_path+"KRW_USDT.csv",index = False, encoding = "cp949")
KRW_EUR.to_csv(save_path+"KRW_EUR.csv",index = False, encoding = "cp949")
KRW_CNY.to_csv(save_path+"KRW_CNY.csv",index = False, encoding = "cp949")
DXY.to_csv(save_path+"DXY.csv",index = False, encoding = "cp949")
KRW_EUR,KRW_EUR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5880 entries, 0 to 5879
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    5880 non-null   datetime64[ns]
 1   Open    5880 non-null   float64       
 2   High    5880 non-null   float64       
 3   Low     5880 non-null   float64       
 4   Close   5880 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 229.8 KB


(           time     Open     High      Low    Close
 0    2000-01-03  1138.67  1159.36  1132.41  1157.32
 1    2000-01-04  1154.05  1161.38  1145.56  1157.46
 2    2000-01-05  1154.87  1180.91  1153.64  1170.41
 3    2000-01-06  1170.29  1194.27  1168.17  1183.39
 4    2000-01-07  1183.13  1191.03  1165.42  1170.66
 ...         ...      ...      ...      ...      ...
 5875 2022-07-14  1313.19  1329.46  1308.95  1318.52
 5876 2022-07-15  1319.95  1336.80  1319.26  1329.20
 5877 2022-07-18  1330.44  1341.04  1326.34  1337.02
 5878 2022-07-19  1337.94  1345.31  1328.27  1334.35
 5879 2022-07-20  1335.43  1346.24  1334.08  1343.25
 
 [5880 rows x 5 columns],
 None)

In [9]:
#데이터 가이드 파일 ,시고저종이 있는 데이터는 good 하지만 없는 데이터는 새로운 컬럼명을 부여할 것!
datagGuide = pd.read_excel("datagGuide.xlsx", sheet_name = None)


def Pretreatment_by_datagGuide(df): #데이터 가이드에서 다운한 데이터 전처리
    data_columns = list(df.keys())
    data = {}
    for i in range(len(data_columns)):
        data[data_columns[i]] = pd.DataFrame(df[data_columns[i]].iloc[13:,:].values,columns = ["time","Open","High","Low","Close"])

        data[data_columns[i]] = data[data_columns[i]].sort_values(['time'],ascending= True).drop_duplicates('time').reset_index(drop = True).rename(columns={'날짜':'time'})
        data[data_columns[i]][["Open","High","Low","Close"]] = data[data_columns[i]][["Open","High","Low","Close"]].astype("float")
    
    return data


#기준금리
base_rate = pd.read_excel("금리.xlsx", sheet_name = None)

def base_rate_by_datagGuide(df):
    
    data_columns = list(df.keys())
    data = {}
    for i in range(len(data_columns)):
        data[data_columns[i]] = pd.DataFrame(df[data_columns[i]].iloc[13:,:].values,columns = ["time","Open","None"])

        data[data_columns[i]] = data[data_columns[i]].sort_values(['time'],ascending= True).drop_duplicates('time').reset_index(drop = True).rename(columns={'날짜':'time'})
        data[data_columns[i]][["Open"]] = data[data_columns[i]][["Open"]].astype("float")
        try:
            del data[data_columns[i]]["None"]
        except:
            pass
    return data


datagGuide = Pretreatment_by_datagGuide(datagGuide)
base_rate = base_rate_by_datagGuide(base_rate)


In [11]:
datagGuide

{'코스피':            time     Open     High      Low    Close
 0    2000-01-03  1028.07  1028.07  1028.07  1028.07
 1    2000-01-04  1028.33  1066.18  1016.59  1059.04
 2    2000-01-05  1006.87  1026.52   984.05   986.31
 3    2000-01-06  1013.95  1014.90   953.50   960.79
 4    2000-01-07   949.17   970.16   930.84   948.65
 ...         ...      ...      ...      ...      ...
 5877 2022-07-13  2324.45  2341.19  2312.49  2328.61
 5878 2022-07-14  2316.41  2338.29  2307.69  2322.32
 5879 2022-07-15  2335.12  2337.30  2293.45  2330.98
 5880 2022-07-18  2350.48  2376.50  2344.12  2375.25
 5881 2022-07-19  2367.52  2372.74  2359.59  2370.97
 
 [5882 rows x 5 columns],
 '프랑스':            time     Open     High      Low    Close
 0    2000-01-03  6024.38  6102.12  5901.77  6024.38
 1    2000-01-04  5922.23  5925.07  5657.20  5922.23
 2    2000-01-05  5521.83  5589.50  5461.59  5521.83
 3    2000-01-06  5485.93  5530.26  5388.85  5485.93
 4    2000-01-07  5423.88  5561.69  5423.88  5539.61
 ...

In [ ]:
keys = list(datagGuide.keys())
for i in range(len(datagGuide)):
    
    print(datagGuide[keys[i]])
    print(datagGuide[keys[i]].info())
    
    datagGuide[keys[i]].to_csv(save_path+keys[i]+".csv",index = False, encoding = "cp949")

keys = list(base_rate.keys())
for i in range(len(base_rate)):
    
    print(base_rate[keys[i]])
    print(base_rate[keys[i]].info())
    
    base_rate[keys[i]].to_csv(save_path+keys[i]+".csv",index = False, encoding = "cp949")

    

In [12]:
base_rate

{'미국금리':            time  Open
 0    1999-07-01  4.75
 1    1999-07-02  5.00
 2    1999-07-03  5.00
 3    1999-07-04  5.00
 4    1999-07-05  5.00
 ...         ...   ...
 8416 2022-07-16  1.75
 8417 2022-07-17  1.75
 8418 2022-07-18  1.75
 8419 2022-07-19  1.75
 8420 2022-07-20  1.75
 
 [8421 rows x 2 columns],
 '한국금리':            time  Open
 0    1999-07-01  4.75
 1    1999-07-02  4.75
 2    1999-07-03  4.75
 3    1999-07-04  4.75
 4    1999-07-05  4.75
 ...         ...   ...
 8416 2022-07-16  2.25
 8417 2022-07-17  2.25
 8418 2022-07-18  2.25
 8419 2022-07-19  2.25
 8420 2022-07-20  2.25
 
 [8421 rows x 2 columns]}

In [674]:
# keys  = list(base_rate.keys())
# for i in range(len(keys)):
    
#     print(base_rate[keys[i]])
#     print(base_rate[keys[i]].info())
#     base_rate[keys[i]].to_csv(save_path+keys[i]+".csv",index = False, encoding = "cp949")

In [675]:
# yes traider 파일
krx_100 = pd.read_csv("krx 100.csv" , encoding = "cp949",names = ['날짜',"Open","High","Low","Close"])
BTC_KRW = pd.read_csv("daydata.csv" , encoding = "cp949",names = ['날짜',"Open","High","Low","Close"])

def Pretreatment_by_yes_trader(df): #데이터 가이드에서 다운한 데이터 전처리
    df["날짜"] = [df["날짜"][i][0:4]+"-"+df["날짜"][i][5:7]+"-"+df["날짜"][i][8:10] for i in range(len(df["날짜"]))]
    df['날짜'] = pd.to_datetime(df['날짜'], format='%Y-%m-%d', errors='raise')
    
    # 예스트레이더는 장이 열려있으면 데이터를 이상하게 주기 떄문에 중복제거, 인덱스 초기화 를 하였음
    df = df.sort_values(['날짜'],ascending= True).drop_duplicates('날짜').reset_index(drop = True).rename(columns={'날짜':'time'})
    
    return df

krx_100 = Pretreatment_by_yes_trader(krx_100)
BTC_KRW = Pretreatment_by_yes_trader(BTC_KRW)

In [676]:
krx_100.to_csv(save_path+"krx_100"+".csv",index = False, encoding = "cp949")
BTC_KRW.to_csv(save_path+"BTC_KRW"+".csv",index = False, encoding = "cp949")

krx_100,krx_100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5326 entries, 0 to 5325
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    5326 non-null   datetime64[ns]
 1   Open    5326 non-null   float64       
 2   High    5326 non-null   float64       
 3   Low     5326 non-null   float64       
 4   Close   5326 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 208.2 KB


(           time     Open     High      Low    Close
 0    2001-01-02  1000.00  1000.00  1000.00  1000.00
 1    2001-01-03   997.82   997.82   997.82   997.82
 2    2001-01-04  1071.24  1071.24  1071.24  1071.24
 3    2001-01-05  1111.68  1111.68  1111.68  1111.68
 4    2001-01-08  1122.48  1122.48  1122.48  1122.48
 ...         ...      ...      ...      ...      ...
 5321 2022-07-14  4792.35  4838.32  4772.45  4800.55
 5322 2022-07-15  4830.37  4836.85  4745.72  4833.14
 5323 2022-07-18  4883.78  4937.47  4870.19  4933.70
 5324 2022-07-19  4911.63  4922.77  4888.12  4920.11
 5325 2022-07-20  5005.08  5014.91  4949.55  4959.15
 
 [5326 rows x 5 columns],
 None)

In [679]:
# 바이낸스 데이터 

from binance.client import Client # python-binance
from binance.spot import Spot #!pip install binance-connector
import pickle
import ccxt
import requests
import time
import requests
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import math
import numpy as np
import matplotlib.pyplot as plt

def bin_sym_names():#바이낸스 심볼명들 도출
    result = requests.get('https://api.binance.com/api/v3/ticker/price')
    js = result.json()
    symbols = [x['symbol'] for x in js]
    symbols_usdt = [x for x in symbols if 'USDT' in x]  # 끝이 USDT로 끝나는 심볼들, ['BTCUSDT', 'ETHUSDT', ...]
    return symbols_usdt

def bin_get_data(start_date, end_date, symbol , interval): # 시작일 , 종료일 , 심볼명 , 분/시간/일/주/월 데이터 선택
    URL = 'https://api.binance.com/api/v3/klines'
    
    COLUMNS = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'quote_av', 'trades', 
               'tb_base_av', 'tb_quote_av', 'ignore']
    data = []
    
    start = int(time.mktime(datetime.strptime(start_date + ' 00:00', '%Y-%m-%d %H:%M').timetuple())) * 1000
    end = int(time.mktime(datetime.strptime(end_date +' 23:59', '%Y-%m-%d %H:%M').timetuple())) * 1000
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': 1000,
        'startTime': start,
        'endTime': end
    }
    
    while start < end:
#         print(datetime.fromtimestamp(start // 1000))
        params['startTime'] = start
        result = requests.get(URL, params = params)
        js = result.json()
        if not js:
            break
        data.extend(js)  # result에 저장
        start = js[-1][0] + 60000  # 다음 step으로
    # 전처리
    if not data:  # 해당 기간에 데이터가 없는 경우
        print('해당 기간에 일치하는 데이터가 없습니다.')
        return -1
    df = pd.DataFrame(data)
    df.columns = COLUMNS
    df['Open_time'] = df.apply(lambda x:datetime.fromtimestamp(x['Open_time'] // 1000), axis=1)
    df = df.drop(columns = ['Close_time', 'ignore'])
    df['Symbol'] = symbol
    df.loc[:, 'Open':'tb_quote_av'] = df.loc[:, 'Open':'tb_quote_av'].astype(float)  # string to float
    df['trades'] = df['trades'].astype(int)
    
############    


    df['날짜'] = pd.to_datetime(df['Open_time'], format='%Y-%m-%d %H:%M:%S', errors='raise')
#     df = df.set_index('Open_time',drop=True)
    df["날짜"] = df["날짜"].astype('str')
    df["날짜"] = [df["날짜"][i][0:4]+"-"+df["날짜"][i][5:7]+"-"+df["날짜"][i][8:10] for i in range(len(df["날짜"]))]
    df['날짜'] = pd.to_datetime(df['날짜'], format='%Y-%m-%d', errors='raise')
    
    df = df.sort_values(['날짜'],ascending= True).drop_duplicates('날짜').reset_index(drop = True).rename(columns={'날짜':'time'})
    del df["Open_time"]
    
    return df

# 바이낸스 상장 심볼명 (현물) 데이터 가져옮
bin_sym_name = bin_sym_names() # bin_sym_name = symbols_usdt


# 시작~끝일 데이터 가져옮
start_date = '2017-01-01' # 시작일 (2017 년도 부터 가능)
end_date = '2022-12-31' # 종료일
interval = "1d"  #  ,'4h',"1m" 
symbol = bin_sym_name[0] # 숫자만 바꾸면 다른 코인 데이터 가져롬
print(symbol)

bin_data = bin_get_data(start_date, end_date, symbol , interval) #바이낸스에서 데이터 가져옮

BTCUSDT


In [680]:
bin_data.to_csv(save_path+"bin_data"+".csv",index = False, encoding = "cp949")
bin_data,bin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Open         1799 non-null   float64       
 1   High         1799 non-null   float64       
 2   Low          1799 non-null   float64       
 3   Close        1799 non-null   float64       
 4   Volume       1799 non-null   float64       
 5   quote_av     1799 non-null   float64       
 6   trades       1799 non-null   int32         
 7   tb_base_av   1799 non-null   float64       
 8   tb_quote_av  1799 non-null   float64       
 9   Symbol       1799 non-null   object        
 10  time         1799 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(8), int32(1), object(1)
memory usage: 147.7+ KB


(          Open      High       Low     Close         Volume      quote_av  \
 0      4261.48   4485.39   4200.74   4285.08     795.150377  3.454770e+06   
 1      4285.08   4371.52   3938.77   4108.37    1199.888264  5.086958e+06   
 2      4108.37   4184.69   3850.00   4139.98     381.309763  1.549484e+06   
 3      4120.98   4211.08   4032.62   4086.29     467.083022  1.930364e+06   
 4      4069.13   4119.62   3911.79   4016.00     691.743060  2.797232e+06   
 ...        ...       ...       ...       ...            ...           ...   
 1794  20830.04  21588.94  20478.61  21195.60  121011.673930  2.539617e+09   
 1795  21195.60  21684.54  20750.01  20798.16  118229.452500  2.511251e+09   
 1796  20799.58  22777.63  20762.45  22432.58  239942.731320  5.263397e+09   
 1797  22432.58  23800.00  21579.54  23396.62  263770.765740  5.954420e+09   
 1798  23398.48  23935.00  22922.00  23819.67  118577.636860  2.786275e+09   
 
        trades     tb_base_av   tb_quote_av   Symbol       tim

In [650]:
# KRW_USDT = Pretreatment_by_invest(KRW_USDT)
# KRW_EUR = Pretreatment_by_invest(KRW_EUR)
# KRW_CNY = Pretreatment_by_invest(KRW_CNY)
# datagGuide = Pretreatment_by_datagGuide(datagGuide)
# base_rate = for_base_rate(base_rate)
# krx_100 = Pretreatment_by_yes_trader(krx_100)
# BTC_KRW = Pretreatment_by_yes_trader(BTC_KRW)
# bin_data = bin_get_data(start_date, end_date, symbol , interval) #바이낸스에서 데이터 가져옮